# **Byte Pair Encoding (BPE) - Uzman**

Önceki çalışmamızda BPE algoritmasının çekirdeğini (istatistik al -> birleştir) sıfırdan inşa ettik. Ancak modern LLM'ler (GPT serisi) bu işlemi ham metin üzerinde doğrudan yapmazlar; verimlilik ve bağlam yönetimi için ek katmanlar kullanırlar.

Bu notebook'ta, gerçek bir LLM tokenizer'ının sahip olduğu gelişmiş özellikleri ve endüstri standartlarını inceleyeceğiz:

1.  **Regex (Düzenli İfadeler):** Metni BPE'ye sokmadan önce mantıksal parçalara (kelime, noktalama, sayı) ayırma stratejisi.
2.  **Special Tokens:** `<|endoftext|>` gibi modelin kontrol sinyallerini yönetme.
3.  **Tiktoken:** OpenAI'ın resmi kütüphanesi ve GPT-2'nin gerçek kelime dağarcığı dosyalarının analizi.

> 🔗 **Kaynaklar ve Referanslar:**
> * **Görselleştirme Aracı:** Metinlerin modeller tarafından nasıl parçalandığını canlı görmek için harika bir araç: [Tiktokenizer App](https://tiktokenizer.vercel.app/)
> * **Orijinal Kaynak Kod:** OpenAI GPT-2 modelinin temelindeki resmi Python uygulaması ve Regex desenleri: [GitHub: GPT-2](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
> * **Derinlemesine İnceleme:** Özel tokenların (Special Tokens) mantığı ve kullanımı üzerine topluluk tartışması: [Reddit: ChatGPT Special Tokens](https://www.reddit.com/r/ChatGPT/comments/14afi5g/chatgpt_special_tokens/)

<p align="center">
  <img src="https://github.com/omertarikyilmaz/byte-pair-encoding/blob/main/assets/cahit_arf.jpg?raw=true" alt="Cahit Arf" width="800">
</p>

In [ ]:
# @title Veri Seti Cahit Arf (Çift Tıklayarak Açabilirsin)
dataset = """Ord. Prof. Dr. Cahit Arf (1910 - 1997), cebir ve sayılar teorisi alanındaki çalışmalarıyla dünya çapında tanınan, "Arf Sabiti", "Arf Halkaları" ve "Arf Kapanışları" gibi matematiksel terimleri bilim dünyasına kazandıran Türk matematikçidir. 1910 yılında Selanik'te doğan Arf, ilkokulu o yıllarda sultani adı verilen liselerin ilk kısmında okumuş, daha sonra Paris'te St. Louis Lisesi'nde eğitimini tamamlamıştır. Yükseköğrenimini Fransa'da Ecole Normale Superieure'de 1932 yılında tamamlayarak Türkiye'ye dönmüştür. Bir süre Galatasaray Lisesi'nde matematik öğretmenliği yaptıktan sonra, İstanbul Üniversitesi Fen Fakültesi'nde doçent adayı olarak çalışmaya başlamıştır. Doktorasını yapmak üzere 1937 yılında Almanya'ya, Göttingen Üniversitesi'ne gitmiş ve burada ünlü matematikçi Helmut Hasse ile çalışmıştır. Hasse'nin "Bu konuda doktora yapan öğrencilerim 1.5 - 2 yılda bitiriyor" dediği zorlu doktora çalışmasını Cahit Arf, 1938 yılında tamamlayarak büyük bir başarıya imza atmıştır. Bu çalışma sonucunda, cisimlerin kuadratik formlarının sınıflandırılmasında ortaya çıkan ve bugün literatürde "Arf Değişmezi" (Arf Invariant) olarak bilinen değişmezi bulmuştur. Bu keşif, karakteristiği 2 olan cisimler üzerindeki kuadratik formlar teorisine köklü bir katkı sağlamıştır. Ayrıca "Hasse-Arf Teoremi" ile matematikte kalıcı bir iz bırakmıştır. Türkiye'ye döndüğünde İstanbul Üniversitesi'nde çalışmaya devam etmiş, 1943'te profesör, 1955'te ordinaryüs profesör unvanını almıştır. 1962 yılında dönemin Cumhurbaşkanı Cemal Gürsel'in atamasıyla TÜBİTAK'ın (Türkiye Bilimsel ve Teknolojik Araştırma Kurumu) kuruluş çalışmalarında kurucu üye olarak yer almış ve uzun yıllar Bilim Kurulu Başkanlığı yapmıştır. Cahit Arf, matematiği sadece bir meslek değil, bir yaşam biçimi ve sanat olarak görmüştür. Onun şu sözü meşhurdur: "Matematik esas olarak sabır olayıdır. Belleyerek değil, keşfederek anlamak gerekir." Robert Kolej'de, ODTÜ'de (Orta Doğu Teknik Üniversitesi) ve yurt dışında Institute for Advanced Study (Princeton) ile California Üniversitesi'nde (Berkeley) misafir öğretim üyesi olarak bulunmuştur. 1948'de İnönü Armağanı'nı, 1974'te TÜBİTAK Bilim Ödülü'nü kazanmıştır. 1980 yılında İTÜ ve Karadeniz Teknik Üniversitesi, 1981 yılında ise ODTÜ kendisine onursal doktora unvanı vermiştir. Bugün kullandığımız 10 Türk Lirası banknotlarının arka yüzünde Cahit Arf'ın portresi ve "Arf Değişmezi" formülünden bir kesit yer almaktadır. Cahit Arf, 26 Aralık 1997'de İstanbul'da geçirdiği kalp rahatsızlığı sonucu hayatını kaybetmiş, ancak geride bıraktığı teoremlerle ölümsüzleşmiştir."""

###**Regex ile Zorunlu Bölme (GPT-2 Deseni)**

Standart BPE algoritmasının en büyük riski, kelimeleri noktalama işaretleriyle birleştirmesidir (örneğin "okul" ve "okul." farklı token olur). GPT-2 bunu önlemek için metni BPE işleminden önce özel bir **Regex (Düzenli İfade)** deseniyle parçalar.

Görselde yer alan ve aşağıda uygulayacağımız bu desen, metni şu kategorilere ayırır:
* Kısaltmalar (İngilizce `'s`, `'t` vb.)
* Kelime Karakterleri (Harfler)
* Sayılar
* Kelimelerden ayrı durması gereken noktalama işaretleri
* Boşluklar

Bu sayede "Cahit Arf (1910)" ifadesi `['Cahit', 'Arf', '(', '1910', ')']` olarak temizce ayrılır.

In [ ]:
import regex as re

gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

print(re.findall(gpt2pat, "Cahit Arf (1910)"))

['Cahit', ' Arf', ' (', '1910', ')']


In [ ]:
tokens = re.findall(gpt2pat, dataset)

print(f"Dataset Karakter Sayısı: {len(dataset)}")
print(f"Regex Parça Sayısı:      {len(tokens)}")
print("-" * 50)

print(tokens[10:50])

Dataset Karakter Sayısı: 2583
Regex Parça Sayısı:      448
--------------------------------------------------
[' -', ' 1997', '),', ' cebir', ' ve', ' sayılar', ' teorisi', ' alanındaki', ' çalışmalarıyla', ' dünya', ' çapında', ' tanınan', ',', ' "', 'Arf', ' Sabiti', '",', ' "', 'Arf', ' Halkaları', '"', ' ve', ' "', 'Arf', ' Kapanışları', '"', ' gibi', ' matematiksel', ' terimleri', ' bilim', ' dünyasına', ' kazandıran', ' Türk', ' matematikçidir', '.', ' 1910', ' yılında', ' Selanik', "'t", 'e']


In [ ]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

In [ ]:
vocab_size = 276
num_merges = vocab_size - 256
ids = list(dataset.encode("utf-8"))

merges = {}
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i

  ids = merge(ids, pair, idx)
  merges[pair] = idx

In [ ]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

In [ ]:
def decode(ids):
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

def encode(text):
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break

    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens